In [2]:
import csv
import os
import glob
import pandas as pd
from nipype.interfaces.base import Bunch

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [22]:
for filename in glob.glob('*/data/*_Scanner_*.log'): #takes all logfiles calculates corrected time values, inserts subject info and combines all logfiles
#     print(filename)

    subID = filename.split('/')[2].split('_')[0]
    task = filename.split('/')[0]
    run = filename.split('/')[2].split('_')[3]

#     print(subID, task, run)
    csvFile = filename.split('/')[-1].replace('log', 'csv')
    logFile = list(csv.reader(open(filename,'r'), delimiter='\t', dialect=csv.excel_tab))

    if len(logFile) > 20:
        # get timing of first scan trigger 
        scanBeginTime = [s for i, s in enumerate(logFile) if 'Keypress: 5' in s][0][0]
        
        i = 0
        expBeginRow = 'empty'
        expBeginTime = 'empty'

        for row in logFile: #Find the point when experiment starts

            if len(row) > 1 and row[1]=='DATA ' and row[2].startswith(('Keypress: 1', 'Keypress: 2')) and expBeginTime == 'empty':
                    expBeginTime=row[0]
                    expBeginRow = i
            i = i + 1

        for k in range(expBeginRow): #delete everything above the point when experiment starts
            logFile.pop(0)

        rewardedTrials = []
        failedTrials = []
        rewardCues = []
        loseCues = []
        neutralCues = []

        for row in logFile:
            if len(row) > 1:
                subtractedTime = float(row[0]) - float(scanBeginTime) #this is the time value that will be used for onsets. 
                row.insert(1,subtractedTime)
                row.insert(0,filename[0:3])
               
                # MID regressors
                if task == 'MID':
                    
                    if 'cueImage: image' in row[4]: #trial started
                        trialStart_onset = row[1].strip()
                        
                    # rewarded trials
                    if 'You won' in row[4] and 'Correct Response!' in row[4]: # rewarded trials
                        rewardedTrials.append(' '.join([row[1].strip(), '.778', '1']))
                    
                    # failed trials    
                    if 'You did not win $' in row[4]:
                        failedTrials.append(' '.join([row[1].strip(), '.778', '1']))

                    # reward anticipation
                    if 'cueImage' in row[4] and 'Win' in row[4]:
                        rewardCues.append(' '.join([row[1].strip(), '.778', '1']))

                    # loss anticipation
                    if 'cueImage' in row[4] and 'Lose' in row[4]:
                        loseCues.append(' '.join([row[1].strip(), '.778', '1']))

                    # neutral cue
                    if 'cueImage' in row[4] and 'Neutral' in row[4]:
                        neutralCues.append(' '.join([row[1].strip(), '.778', '1']))
                        
                elif task == 'SST': 
                # SST regressors
                    # failed stop
                    # correct stop
                    # correct go
                    

        # write out evFiles
        if task == 'MID':

            with open(os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_rewardedTrials.txt'), 'w') as f:
                for item in rewardedTrials:
                    f.write("%s\n" % item)

            with open(os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_failedTrials.txt'), 'w') as f:
                for item in failedTrials:
                    f.write("%s\n" % item)

            with open(os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_rewardCues.txt'), 'w') as f:
                for item in rewardCues:
                    f.write("%s\n" % item)

            with open(os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_loseCues.txt'), 'w') as f:
                for item in loseCues:
                    f.write("%s\n" % item)

            with open(os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_neutralCues.txt'), 'w') as f:
                for item in neutralCues:
                    f.write("%s\n" % item)

    else:
        print('Error: not enough rows in', filename) # incomplete dataset



Error: not enough rows in MID/data/034_MID_Scanner_run-1_2019_Aug_27_0918.log
Error: not enough rows in MID/data/037_MID_Scanner_run-1_2019_Aug_23_1244.log
Error: not enough rows in MID/data/038_MID_Scanner_run-1_2019_Aug_25_1121.log
Error: not enough rows in SST/data/025_SST_Scanner_run-2_2019_Sep_02_0929.log
Error: not enough rows in SST/data/025_SST_Scanner_run-2_2019_Sep_02_0930.log


In [326]:
for filename in glob.glob('*/data/*_Scanner_*.log'): #takes all logfiles calculates corrected time values, inserts subject info and combines all logfiles

    logFile = list(csv.reader(open(filename,'r'), delimiter='\t', dialect=csv.excel_tab))
    task = filename.split('/')[0]
    
    if len(logFile) > 20 and task == 'MID':
        print(filename, len(logFile))

        csvFile = pd.read_csv(filename.replace('log', 'csv'))
        probeResp_list = csvFile['probeResp.corr'].tolist()

        subID = filename.split('/')[2].split('_')[0]
        subID=f'{int(subID):03}'
        print(subID)


        run = filename.split('/')[2].split('_')[3]
        run = run[:4] + '0' + run[4:] # add zero in run
    
        # get timing of first scan trigger 
        scanBeginTime = [s for i, s in enumerate(logFile) if 'Keypress: 5' in s][0][0]
        
        i = 0
        expBeginRow = 'empty'
        expBeginTime = 'empty'

        trial_counter = 0

        cond_list = []
        cue_onsets = []
        fixation_onsets = []
        target_onsets = []
        feedback_onsets = []

        cue_durs = []
        fixation_durs = []
        feedback_durs = []

        for row in logFile:
            if len(row) > 1:
                subtractedTime = float(row[0]) - float(scanBeginTime) #this is the time value that will be used for onsets. 
                row.insert(1,subtractedTime)
                row.insert(0,filename[0:3])
               
                # MID regressors
                if task == 'MID':
        
                    if 'cueImage: image' in row[4]: #trial started
                        trial_counter = trial_counter + 1           
                        cue_onsets.append(float(row[1].strip()))

                        # get trial condition
                        cond_list.append(row[4].split('/', 1)[-1].split('.', 1)[0])

                        if trial_counter > 1:
                            feedback_durs.append(float(cue_onsets[trial_counter - 1] - feedback_onsets[trial_counter - 2]))

                    if 'fixation: autoDraw = True' in row[4]:
                        fixation_onsets.append(float(row[1].strip()))
                        cue_durs.append(float(fixation_onsets[trial_counter - 1] - cue_onsets[trial_counter - 1]))

                    if 'Target: image' in row[4]:
                        target_onsets.append(float(row[1].strip()))
                        fixation_durs.append(float(target_onsets[trial_counter - 1] - fixation_onsets[trial_counter - 1]))

                    if 'feedbackMsg: text =' in row[4]:
                        feedback_onsets.append(float(row[1].strip()))

                    if 'polygon: autoDraw = True' in row[4]: # for last trial, calculate feedback dur
                        feedback_durs.append(float(row[1].strip()) - feedback_onsets[trial_counter - 1])
                        
#                 elif task == 'SST':
                    
        # cue regressors        
        df = pd.DataFrame(list(zip(cond_list, cue_onsets, cue_durs, probeResp_list)), columns = ['condition', 'onset', 'duration', 'probeResp.corr'])
        df['amplitude'] = 1


        win_cues = df[df['condition'].str.contains("Win")].drop(['condition', 'probeResp.corr'], axis = 1)
        lose_cues = df[df['condition'].str.contains("Lose")].drop(['condition', 'probeResp.corr'], axis = 1)
        neut_cues = df[df['condition'].str.contains("Neutral")].drop(['condition', 'probeResp.corr'], axis = 1)

        fname = os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_win_cues.txt')
        np.savetxt(fname, win_cues.values, fmt='%.3f')

        fname = os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_lose_cues.txt')
        np.savetxt(fname, lose_cues.values, fmt='%.3f')

        fname = os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_neut_cues.txt')
        np.savetxt(fname, neut_cues.values, fmt='%.3f')

        # fixation regressors
        df = pd.DataFrame(list(zip(cond_list, fixation_onsets, fixation_durs, probeResp_list)), columns = ['condition', 'onset', 'duration', 'probeResp.corr'])
        df['amplitude'] = 1

        win_fixation = df[df['condition'].str.contains("Win")].drop(['condition', 'probeResp.corr'], axis = 1)
        lose_fixation = df[df['condition'].str.contains("Lose")].drop(['condition', 'probeResp.corr'], axis = 1)
        neut_fixation = df[df['condition'].str.contains("Neutral")].drop(['condition', 'probeResp.corr'], axis = 1)

        fname = os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_win_fixation.txt')
        np.savetxt(fname, win_fixation.values, fmt='%.3f')

        fname = os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_lose_fixation.txt')
        np.savetxt(fname, lose_fixation.values, fmt='%.3f')

        fname = os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_neut_fixation.txt')
        np.savetxt(fname, neut_fixation.values, fmt='%.3f')

        # feedback regressors
        df = pd.DataFrame(list(zip(cond_list, feedback_onsets, feedback_durs, probeResp_list)), columns = ['condition', 'onset', 'duration', 'probeResp.corr'])
        df['amplitude']=1

        win_feedback = df[df['condition'].str.contains("Win")].drop(['condition', 'probeResp.corr'], axis = 1)
        lose_feedback = df[df['condition'].str.contains("Lose")].drop(['condition', 'probeResp.corr'], axis = 1)
        neut_feedback = df[df['condition'].str.contains("Neutral")].drop(['condition', 'probeResp.corr'], axis = 1)

        fname = os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_win_feedback.txt')
        np.savetxt(fname, win_feedback.values, fmt='%.3f')

        fname = os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_lose_feedback.txt')
        np.savetxt(fname, lose_feedback.values, fmt='%.3f')

        fname = os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_neut_feedback.txt')
        np.savetxt(fname, neut_feedback.values, fmt='%.3f')

        # hits vs misses
        rewarded_trials = df.loc[df ['probeResp.corr'] == 1.0 ][df['condition'].str.contains('Win')].drop(['condition', 'probeResp.corr'], axis = 1)
        failed_reward = df.loc[df ['probeResp.corr'] == 0 ][df['condition'].str.contains('Win')].drop(['condition', 'probeResp.corr'], axis = 1)

        fname = os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_rewarded_trials.txt')
        np.savetxt(fname, rewarded_trials.values, fmt='%.3f')

        fname = os.path.join(os.getcwd(), task, 'evFiles', subID + '_' + task + '_' + run + '_failed_reward.txt')
        np.savetxt(fname, failed_reward.values, fmt='%.3f')

        # concat hits and miss on reward trials
        trialInfo = pd.concat([df.loc[df ['probeResp.corr'] == 1.0 ][df['condition'].str.contains('Win')], df.loc[df ['probeResp.corr'] == 0 ][df['condition'].str.contains('Win')]], ignore_index=True)

        trialInfo.loc[(trialInfo['probeResp.corr'] == 1.0),'trial_type'] = 'hit'
        trialInfo.loc[(trialInfo['probeResp.corr'] == 0.0),'trial_type'] = 'miss'

        fname=os.path.join(os.getcwd(), task, 'evFiles', 'sub-ULG' + subID + '_ses-0_task-' + task + '_' + run + '_events.tsv')
        trialInfo.to_csv(fname, sep='\t', index=False)
    
    elif len(logFile) < 20:
        print('Error: not enough rows in', filename) # incomplete dataset

MID/data/001_MID_Scanner_run-1_2019_Sep_06_1102.log 1293
001
MID/data/001_MID_Scanner_run-2_2019_Sep_06_1108.log 1287
001


/Users/tbw665/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/tbw665/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:128: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/tbw665/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:137: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


MID/data/002_MID_Scanner_run-1_2019_Sep_05_1113.log 1286
002
MID/data/002_MID_Scanner_run-2_2019_Sep_05_1119.log 1280
002
MID/data/003_MID_Scanner_run-1_2019_Sep_07_1130.log 1286
003
MID/data/003_MID_Scanner_run-2_2019_Sep_07_1137.log 1273
003
MID/data/0042_MID_Scanner_run-1_2019_Aug_21_1231.log 1284
042
MID/data/0042_MID_Scanner_run-2_2019_Aug_21_1238.log 1285
042
MID/data/004_MID_Scanner_run-1_2019_Sep_07_0930.log 1288
004
MID/data/004_MID_Scanner_run-2_2019_Sep_07_0936.log 1278
004
MID/data/006_MID_Scanner_run-1_2019_Sep_08_1100.log 1286
006
MID/data/006_MID_Scanner_run-2_2019_Sep_08_1106.log 1282
006
MID/data/009_MID_Scanner_run-1_2019_Sep_06_1248.log 1286
009
MID/data/009_MID_Scanner_run-2_2019_Sep_06_1254.log 1278
009
MID/data/010_MID_Scanner_run-1_2019_Sep_07_1259.log 1281
010
MID/data/010_MID_Scanner_run-2_2019_Sep_07_1305.log 1273
010
MID/data/011_MID_Scanner_run-1_2019_Sep_07_1505.log 1284
011
MID/data/011_MID_Scanner_run-2_2019_Sep_07_1512.log 1278
011
MID/data/012_MID_Scann

In [317]:
filename='MID/data/034_MID_Scanner_run-1_2019_Aug_27_0918.log'
len(list(csv.reader(open(filename,'r'), delimiter='\t', dialect=csv.excel_tab)))
# pd.read_csv(filename.replace('log', 'csv'))['probeResp.corr'].tolist()


11

In [272]:
conditions = sorted(list(set(trialInfo.trial_type)))
conditions

['hit', 'miss']

In [274]:
# extract onset times and stimulus durations for each condition
onsets = []
durations = []

for itrial in conditions:
    onsets.append(list(trialInfo[trialInfo.trial_type==itrial].Onset))
    durations.append(list(trialInfo[trialInfo.trial_type==itrial].Dur))
    
onsets

[[81.0389,
  132.292,
  149.2128,
  162.2129,
  175.1309,
  212.4656,
  269.9232,
  301.3843,
  307.3427,
  320.7605,
  332.2607],
 [33.9536,
  46.4576,
  100.5021,
  114.3756,
  121.3338,
  137.7503,
  199.4631,
  217.9655,
  225.9206]]

In [277]:
# combine into bunch object
subject_info = [Bunch(conditions=conditions,
                      onsets=onsets,
                      durations=durations,
                      )]
subject_info

[Bunch(conditions=['hit', 'miss'],
       durations=[[1.542100000000005,
         1.502600000000001,
         1.4968000000000075,
         1.4972999999999956,
         1.4976000000000056,
         1.5431000000000097,
         1.4993000000000052,
         1.4992000000000303,
         1.4978000000000407,
         1.5394000000000005,
         1.540200000000027],
        [1.6265999999999963,
         1.6216000000000008,
         1.4956999999999994,
         1.5420999999999907,
         1.5409000000000077,
         1.5001999999999782,
         1.539999999999992,
         1.4971000000000174,
         1.541799999999995]],
       onsets=[[81.0389,
         132.292,
         149.2128,
         162.2129,
         175.1309,
         212.4656,
         269.9232,
         301.3843,
         307.3427,
         320.7605,
         332.2607],
        [33.9536,
         46.4576,
         100.5021,
         114.3756,
         121.3338,
         137.7503,
         199.4631,
         217.9655,
         225

In [ ]:
## Defining contrasts
c1 = ['hit', 'T', conditions, [1, 0]]
c2 = ['miss', 'T', conditions, [0, 1]]
c3 = ['hit > miss', 'T', conditions, [1, -1]]

contrast_list = ['c1', 'c2', 'c3']


In [ ]:
TR = 0.778


In [ ]:
# data input
from nipype import SelectFiles

# directory where preprocessed fMRI data is located
baseDir = '/corral-secure/utdmc/Dell-Medical-Imaging/ULG/BIDS/'

# Output directory
outDir = os.path.join(dataDir,'derivatives')

# String template with {}-based strings
templates = {'anat': 'sub-ULG{subject_id}/ses-0/anat/'
                     'sub-ULG{subject_id}_ses-test_T1w.nii.gz',
             'func': 'sub-ULG{subject_id}/ses-0/func/'
                     'sub-ULG{subject_id}_ses-0_task-{task_id}_run-{run_id}_dir-AP_bold.nii.gz'}

# Create SelectFiles node
sf = Node(SelectFiles(templates,
                      sort_filelist=True),
          name='selectfiles')

# list of values for the iterables
subject_list = [s.split('ULG')[-1] for s in glob.glob(os.path.join(baseDir, 'sub-*'))]
run_list = ['01', '02']
task_list = ['MID', 'SST']

sf.iterables = [('subject_id', subject_list),
                ('run_id', run_list),
                ('task_id', task_list)]

In [ ]:
# model specification
modelspec = Node(modelgen.SpecifyModel(subject_info=subject_info,
                                       input_units='secs',
                                       time_repetition=TR,
                                       high_pass_filter_cutoff=100),
                 name="modelspec")

In [ ]:
# first-level design
level1design = Node(fsl.Level1Design(bases={'dgamma':{'derivs': True}},
                                     interscan_interval=TR,
                                     model_serial_correlations=True,
                                     contrasts=contrast_list),
                    name="level1design")

In [ ]:
# create all the other files necessary to run the model
modelgen = Node(fsl.FEATModel(),
                name='modelgen')

In [ ]:
# create FEAT node
feat = Node(fsl.FEAT(),
            name="feat")

In [ ]:
# create datasink to collect outputs
datasink = Node(DataSink(base_directory=outDir),
                name='datasink')

## Use the following DataSink output substitutions
substitutions = [('_subject_id_', 'sub-'),
                 ('_task_id_', 'task-'),
                 ('_run_id_', 'run-')
                 ]

datasink.inputs.substitutions = substitutions

In [ ]:
# create the workflow
firstLevel = Workflow(name="Level1", base_dir=outDir)

# connect nodes
firstLevel.connect(sf, 'func', modelspec, 'functional_runs')
firstLevel.connect(modelspec, 'session_info', level1design, 'session_info')
firstLevel.connect(level1design, 'fsf_files', modelgen, 'fsf_file')
firstLevel.connect(level1design, 'ev_files', modelgen, 'ev_files')
firstLevel.connect(level1design, 'fsf_files', feat, 'fsf_file')
firstLevel.connect(feat, 'feat_dir', datasink, 'feat_dir')


In [ ]:
# writing out graphs
firstLevel.write_graph(graph2use='orig', dotfilename='graph_orig.dot')

# showing the graph
plt.figure(figsize=[6,6])
img=mpimg.imread(os.path.join(outDir,"Level1_FingerFootLips","graph_orig.png"))
imgplot = plt.imshow(img)
plt.axis('off')
plt.show()



In [ ]:
# running the workflow
firstLevel.run('MultiProc', plugin_args={'n_procs': 10})

In [31]:
# RT = 0.572655256954021
# first trial onset = 33.2480
# keypress in log = 33.8198
33.8198 - 33.2480 

0.5718000000000032

In [32]:
0.572655256954021 - 0.5718000000000032

0.0008552569540177801

In [9]:
pd.read_csv('SST/data/012_SST_Scanner_run-2_2019_Sep_01_1141.csv')

,trial_type,arrow_1,arrow_2,corrAns,jitter,run_end_fixation,trials.thisRepN,trials.thisTrialN,trials.thisN,trials.thisIndex,randomized TrialOrder,key_response.keys,key_response.corr,key_response.rt,trial_t.routineEndVal,ss_delay,iti_duration,SOA,participant,prac_or_scan,handedness,run,date,expName,psychopyVersion,frameRate,Unnamed: 26
0,go,sst_images/Left_Arrow.bmp,None,1,936,None,0,0,0,0,10.0,1,1,0.572655,go,0.05,0.936,1.936000,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
1,go,sst_images/Right_Arrow.bmp,None,2,708,None,0,1,1,1,NaN,2,1,0.422063,go,0.05,0.708,1.708000,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
2,go,sst_images/Right_Arrow.bmp,None,2,728,None,0,2,2,2,NaN,2,1,0.570333,go,0.05,0.728,1.728000,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
3,go,sst_images/Left_Arrow.bmp,None,1,940,None,0,3,3,3,NaN,1,1,0.489716,go,0.05,0.940,1.940000,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
4,go,sst_images/Left_Arrow.bmp,None,1,778,None,0,4,4,4,NaN,1,1,0.404709,go,0.05,0.778,1.778000,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
5,stop,sst_images/Right_Arrow.bmp,sst_images/Stop_Arrow.bmp,none,761,None,0,5,5,5,NaN,None,1,NaN,stop,0.10,0.761,1.761000,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
6,go,sst_images/Right_Arrow.bmp,None,2,2000,None,0,6,6,6,NaN,1,0,0.371129,go,0.10,2.000,3.000000,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
7,go,sst_images/Left_Arrow.bmp,None,1,746,None,0,7,7,7,NaN,1,1,0.336090,go,0.10,0.746,1.746000,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
8,go,sst_images/Right_Arrow.bmp,None,2,1248,None,0,8,8,8,NaN,2,1,0.404158,go,0.10,1.248,2.248000,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
9,stop,sst_images/Left_Arrow.bmp,sst_images/Stop_Arrow.bmp,none,1149,None,0,9,9,9,NaN,None,1,NaN,stop,0.15,1.149,2.149000,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN


In [14]:
logFile

[['12.1147 ',
  'WARNING ',
  'User requested fullscreen with size [1440  900], but screen is actually [1920, 1080]. Using actual size'],
 ['12.6828 ',
  'EXP ',
  "Created window1 = Window(allowGUI=False, allowStencil=False, args=UNKNOWN, autoLog=True, bitsMode=UNKNOWN, blendMode='avg', color=array([0., 0., 0.]), colorSpace='rgb', fullscr=<method-wrapper '__getattribute__' of attributeSetter object at 0x1347abc18>, gamma=None, kwargs=UNKNOWN, lms=UNKNOWN, monitor=<psychopy.monitors.calibTools.Monitor object at 0x1348e1208>, multiSample=False, name='window1', numSamples=2, pos=[0.0, 0.0], screen=0, size=array([1920, 1080]), stereo=False, units='norm', useFBO=True, useRetina=False, viewOri=0.0, viewPos=None, viewScale=None, waitBlanking=True, winType='pyglet')"],
 ['12.6828 ', 'EXP ', 'window1: recordFrameIntervals = False'],
 ['12.8487 ', 'EXP ', 'window1: recordFrameIntervals = True'],
 ['13.0322 ', 'EXP ', 'window1: recordFrameIntervals = False'],
 ['13.4542 ',
  'EXP ',
  "Created r

In [11]:
import numpy as np
import pandas as pd
import csv

# filename = 'SST/data/012_SST_Scanner_run-2_2019_Sep_01_1141.log'

for filename in glob.glob('SST/data/*_Scanner_*.log'): #takes all logfiles calculates corrected time values, inserts subject info and combines all logfiles

    task = 'SST'
    print(filename, len(list(csv.reader(open(filename,'r'), delimiter='\t', dialect=csv.excel_tab))))

    if len(list(csv.reader(open(filename,'r'), delimiter='\t', dialect=csv.excel_tab))) > 50:
        
        run = filename.split('/')[2].split('_')[3]
        run = run[:4] + '0' + run[4:] # add zero in run
    
           

        logFile = list(csv.reader(open(filename,'r'), delimiter='\t', dialect=csv.excel_tab))

        trial_onsets = []
        respList = []

        subID = filename.split('/')[2].split('_')[0]
        subID=f'{int(subID):03}'
        trial_counter = 0 

        # get timing of first scan trigger 
        scanBeginTime = [s for i, s in enumerate(logFile) if 'Keypress: 5' in s][0][0]

        for row in logFile:
            if len(row) > 1:
                csvFile = pd.read_csv(filename.replace('log', 'csv'))

                probeResp_list = csvFile['key_response.corr'].tolist() 
                subtractedTime = float(row[0]) - float(scanBeginTime) #this is the time value that will be used for onsets. 

                row.insert(1,subtractedTime)
                row.insert(0,filename[0:3])


                # list of trial starts
                if 'arrow_key: image =' in row[4]:
                    trial_counter = trial_counter + 1 
                    trial_onsets.append(row[2])

                # make a list of key responses
                if 'Keypress: 1' in row[4] or 'Keypress: 2' in row[4]:    
                    respList.append(row[0])

        csvFile['onset'] = pd.DataFrame(np.array([trial_onsets]).T)

        # SST regressors

        # failed stop
        csvFile.loc[(csvFile['key_response.corr'] == 0) & (csvFile['trial_type'] == 'stop'),'trial_type'] = 'failed_stop' 

        # correct stop
        csvFile.loc[(csvFile['key_response.corr'] == 1) & (csvFile['trial_type'] == 'stop'),'trial_type'] = 'correct_stop'

        # correct go
        csvFile.loc[(csvFile['key_response.corr'] == 1) & (csvFile['trial_type'] == 'go'),'trial_type'] = 'correct_go'

        # failed go
        csvFile.loc[(csvFile['key_response.corr'] == 0) & (csvFile['trial_type'] == 'go'),'trial_type'] = 'failed_go'

        csvFile['duration'] = 1
        csvFile['amplitude'] = 1
        
        corrstop_vs_failstop = csvFile.loc[(csvFile['trial_type'] == 'correct_stop') | (csvFile['trial_type'] == 'failed_stop')]
        corrstop_vs_correctgo = csvFile.loc[(csvFile['trial_type'] == 'correct_stop') | (csvFile['trial_type'] == 'correct_go')]
        
        csvFile = csvFile[['trial_type', 'onset', 'duration', 'key_response.corr']]

        fname=os.path.join('~/remote/corral/ULG/BIDS/derivatives/', 'events', 'sub-ULG' + subID + '_ses-0_task-' + task + '_' + run + 'corrstop_vs_failstop_events.tsv')
        corrstop_vs_failstop.to_csv(fname, sep='\t', index=False)
        
        fname=os.path.join('~/remote/corral/ULG/BIDS/derivatives/', 'events', 'sub-ULG' + subID + '_ses-0_task-' + task + '_' + run + 'corrstop_vs_correctgo.tsv')
        corrstop_vs_correctgo.to_csv(fname, sep='\t', index=False)
    elif len(logFile) < 20:
        print('Error: not enough rows in', filename) # incomplete dataset

SST/data/001_SST_Scanner_run-1_2019_Sep_06_1045.log 2521
SST/data/001_SST_Scanner_run-2_2019_Sep_06_1052.log 2518
SST/data/002_SST_Scanner_run-1_2019_Sep_05_1057.log 2529
SST/data/002_SST_Scanner_run-2_2019_Sep_05_1104.log 2519
SST/data/003_SST_Scanner_run-1_2019_Sep_07_1114.log 2520
SST/data/003_SST_Scanner_run-2_2019_Sep_07_1121.log 2514
SST/data/0042_SST_Scanner_run-1_2019_Aug_21_1246.log 2526
SST/data/0042_SST_Scanner_run-2_2019_Aug_21_1254.log 2517
SST/data/004_SST_Scanner_run-1_2019_Sep_07_0945.log 2518
SST/data/004_SST_Scanner_run-2_2019_Sep_07_0952.log 2520
SST/data/006_SST_Scanner_run-1_2019_Sep_08_1115.log 2527
SST/data/006_SST_Scanner_run-2_2019_Sep_08_1122.log 2520
SST/data/009_SST_Scanner_run-1_2019_Sep_06_1231.log 2525
SST/data/009_SST_Scanner_run-2_2019_Sep_06_1238.log 2519
SST/data/010_SST_Scanner_run-1_2019_Sep_07_1313.log 1011
SST/data/010_SST_Scanner_run-1_2019_Sep_07_1317.log 2538
SST/data/010_SST_Scanner_run-2_2019_Sep_07_1323.log 2553
SST/data/011_SST_Scanner_run-

In [8]:
csvFile
corrgo_vs_failgo = csvFile.loc[(csvFile['trial_type'] == 'correct go') | (csvFile['trial_type'] == 'failed go')] 
corrgo_vs_failgo


,trial_type,onset,duration,key_response.corr
0,correct go,0.0572,1,1
1,correct go,2.1408,1,1
2,correct go,3.9239,1,1
3,correct go,5.9762,1,1
4,correct go,7.7583,1,1
5,correct go,10.1430,1,1
6,correct go,11.9751,1,1
8,correct go,15.8925,1,1
9,correct go,17.9423,1,1
10,correct go,19.8097,1,1


In [72]:


csvFile = csvFile[['trial_type', 'onset', 'key_response.corr']]
failed_stop = csvFile.loc[(csvFile['key_response.corr'] == 0) & (csvFile['trial_type'] == 'stop')]
correct_go = csvFile.loc[(csvFile['key_response.corr'] == 1) & (csvFile['trial_type'] == 'go')]
correct_stop = csvFile.loc[(csvFile['key_response.corr'] == 1) & (csvFile['trial_type'] == 'stop')]

# csvFile.loc[csvFile ['key_response.corr'] == 0 ][csvFile['trial_type'].str.contains('stop')].drop(['trial_type', 'key_response.corr'], axis = 1)
#         failed_reward = df.loc[df ['probeResp.corr'] == 0 ][df['condition'].str.contains('Win')].drop(['condition', 'probeResp.corr'], axis = 1)


,trial_type,onset,key_response.corr
0,go,0.0610,1
1,go,2.0274,1
2,go,3.7614,1
3,go,5.5108,1
4,go,7.4784,1
5,stop,9.2958,1
6,go,11.0789,0
7,go,14.1128,1
8,go,15.8958,1
9,stop,18.1815,1


In [ ]:
cols = list(csvFile)

# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('onset')))
cols

# use ix to reorder
csvFile = csvFile.loc[:, cols]

csvFile['onset'] = pd.to_numeric(csvFile['onset'])

csvFile['onset_plus_rt'] = csvFile['onset'] + csvFile['key_response.rt']

In [41]:
cols

['onset',
 'trial_type',
 'arrow_1',
 'arrow_2',
 'corrAns',
 'jitter',
 'run_end_fixation',
 'trials.thisRepN',
 'trials.thisTrialN',
 'trials.thisN',
 'trials.thisIndex',
 'randomized TrialOrder',
 'key_response.keys',
 'key_response.corr',
 'key_response.rt',
 'trial_t.routineEndVal',
 'ss_delay',
 'iti_duration',
 'SOA',
 'participant',
 'prac_or_scan',
 'handedness',
 'run',
 'date',
 'expName',
 'psychopyVersion',
 'frameRate',
 'Unnamed: 26']

In [49]:
# correct go
# csvFile.loc[(csvFile['key_response.corr'] == 1) & (csvFile['trial_type'] == 'go')]

# failed stop
csvFile.loc[(csvFile['key_response.corr'] == 0) & (csvFile['trial_type'] == 'stop')]

# correct stop
# csvFile.loc[(csvFile['key_response.corr'] == 1) & (csvFile['trial_type'] == 'stop')]

,onset,trial_type,arrow_1,arrow_2,corrAns,jitter,run_end_fixation,trials.thisRepN,trials.thisTrialN,trials.thisN,trials.thisIndex,randomized TrialOrder,key_response.keys,key_response.corr,key_response.rt,trial_t.routineEndVal,ss_delay,iti_duration,SOA,participant,prac_or_scan,handedness,run,date,expName,psychopyVersion,frameRate,Unnamed: 26
27,88.0063,stop,sst_images/Right_Arrow.bmp,sst_images/Stop_Arrow.bmp,none,758,None,0,27,27,27,NaN,2,0,0.437486,stop,0.15,0.758,1.820514,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
33,99.0242,stop,sst_images/Right_Arrow.bmp,sst_images/Stop_Arrow.bmp,none,1026,None,0,33,33,33,NaN,2,0,0.387922,stop,0.10,1.026,2.088078,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
41,114.4253,stop,sst_images/Left_Arrow.bmp,sst_images/Stop_Arrow.bmp,none,862,None,0,41,41,41,NaN,1,0,0.370807,stop,0.05,0.862,1.891193,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
57,144.0425,stop,sst_images/Right_Arrow.bmp,sst_images/Stop_Arrow.bmp,none,702,None,0,57,57,57,NaN,1,0,0.369575,stop,0.15,0.702,1.832425,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
61,151.7934,stop,sst_images/Left_Arrow.bmp,sst_images/Stop_Arrow.bmp,none,1069,None,0,61,61,61,NaN,1,0,0.253643,stop,0.10,1.069,2.265357,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
70,168.9450,stop,sst_images/Right_Arrow.bmp,sst_images/Stop_Arrow.bmp,none,1098,None,0,70,70,70,NaN,2,0,0.288063,stop,0.10,1.098,2.259937,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
80,188.3132,stop,sst_images/Left_Arrow.bmp,sst_images/Stop_Arrow.bmp,none,968,None,0,80,80,80,NaN,1,0,0.338265,stop,0.05,0.968,2.029735,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
118,258.4974,stop,sst_images/Right_Arrow.bmp,sst_images/Stop_Arrow.bmp,none,950,None,0,118,118,118,NaN,2,0,0.386019,stop,0.20,0.950,2.113981,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
130,282.4845,stop,sst_images/Left_Arrow.bmp,sst_images/Stop_Arrow.bmp,none,775,None,0,130,130,130,NaN,1,0,0.472362,stop,0.20,0.775,1.852638,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
135,292.6178,stop,sst_images/Right_Arrow.bmp,sst_images/Stop_Arrow.bmp,none,940,None,0,135,135,135,NaN,2,0,0.322185,stop,0.15,0.940,2.117815,12,Scanner,R,2,2019_Sep_01_1141,SST,3.0.7,60.295805,NaN
